In [1]:
import numpy as np
import pandas as pd
import sys
import mlflow
import warnings
import pickle
from datetime import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import ticker

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import os

In [2]:
%matplotlib inline
warnings.filterwarnings("ignore")

### Only the Date and Sales columns will be used for LSTM

In [3]:
train_store_df = pd.read_csv('data/train_store_merged.csv', parse_dates = True, low_memory = False)

In [4]:
train_store_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,SalePerCustomer,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,...,9.482883,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,1,4,2015-07-30,5020,546,1,1,0,1,2015,...,9.194139,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
2,1,3,2015-07-29,4782,523,1,1,0,1,2015,...,9.143403,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
3,1,2,2015-07-28,5011,560,1,1,0,1,2015,...,8.948214,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
4,1,1,2015-07-27,6102,612,1,1,0,1,2015,...,9.970588,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0


## Isolate the Rossmann Store Sales dataset into time series data

In [5]:
data = train_store_df.groupby("Date").agg({"Sales": "mean"})
data

,Sales
Date,
2013-01-01,87.284560
2013-01-02,6233.030493
2013-01-03,5693.112108
2013-01-04,5954.218834
2013-01-05,5337.751570
...,...
2015-07-27,9602.952466
2015-07-28,8174.953363
2015-07-29,7623.284305


In [6]:
scaler = StandardScaler()
scaled_array = scaler.fit_transform(data)
data['DataScaled'] = scaled_array

In [9]:
SIZE = len(data.DataScaled)
WINDOW_SIZE = 48
BATCH_SIZE = SIZE - WINDOW_SIZE * 2
EPOCHS = 200

In [10]:
DateTrain = data.index.values[0:BATCH_SIZE]
DateValid = data.index.values[BATCH_SIZE:]
XTrain = data.DataScaled.values[0:BATCH_SIZE].astype('float32')
XValid = data.DataScaled.values[BATCH_SIZE:].astype('float32')

# Obtain shapes for vectors of size (,1) for dates series

DateTrain = np.reshape(DateTrain, (-1, 1))
DateValid = np.reshape(DateValid, (-1, 1))

print("Shape of the training set date series: ", DateTrain.shape)
print("Shape of the validation set date series: ", DateValid.shape)
print()
print("Shape of the training set logarithm of sales series: ", XTrain.shape)
print("Shape of the validation set logarithm of sales series in a stateless LSTM: ", XValid.shape)

Shape of the training set date series:  (846, 1)
Shape of the validation set date series:  (96, 1)

Shape of the training set logarithm of sales series:  (846,)
Shape of the validation set logarithm of sales series in a stateless LSTM:  (96,)


In [11]:
tf.random.set_seed(1234)
# add extra dimension
series = tf.expand_dims(XTrain, axis=-1)
series.shape

TensorShape([846, 1])

In [12]:
# create tensor from each individual element
dataset = tf.data.Dataset.from_tensor_slices(series)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(1,), dtype=tf.float32, name=None)>

In [13]:
# takes a window_size + 1 chunk from the slices
dataset = dataset.window(WINDOW_SIZE + 1, shift=1, drop_remainder=True)

In [14]:
# Example of Window
datasetEx = tf.data.Dataset.from_tensor_slices(tf.range(10))
datasetEx = datasetEx.window(5, shift=1, drop_remainder=True)
for window in datasetEx:
    print([elem.numpy() for elem in window])

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


In [15]:
dataset = dataset.flat_map(lambda window: window.batch(WINDOW_SIZE + 1))

In [16]:
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))

In [15]:
dataset = dataset.batch(BATCH_SIZE).prefetch(1)

In [18]:
def windowed_dataset(series, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE):
  series = tf.expand_dims(series, axis=-1)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [19]:
DatasetTrain = windowed_dataset(XTrain)
DatasetVal = windowed_dataset(XValid)

# Model Training

In [20]:
model = Sequential()
model.add(LSTM(8, input_shape=[None, 1], return_sequences=True))
model.add(LSTM(4, input_shape=[None, 1]))
model.add(Dense(1))
model.compile(loss="huber_loss", optimizer='adam')

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 8)           320       
                                                                 
 lstm_1 (LSTM)               (None, 4)                 208       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 533 (2.08 KB)
Trainable params: 533 (2.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
os.environ['GIT_PYTHON_REFRESH'] = 'quiet'
# Set the experiment name
mlflow.set_experiment("LSTM")

# Start a new run with a descriptive name
with mlflow.start_run(run_name="LSTM_model"):

    # Enable automatic logging for TensorFlow
    mlflow.tensorflow.autolog()

    # Train your model
    history = model.fit(DatasetTrain, epochs=EPOCHS, validation_data=DatasetVal, verbose=1)


2024/02/05 23:50:20 INFO mlflow.tracking.fluent: Experiment with name 'LSTM' does not exist. Creating a new experiment.


2024/02/05 23:50:21 INFO mlflow.types.utils: MLflow 2.9.0 introduces model signature with new data types for lists and dictionaries. For input such as Dict[str, Union[scalars, List, Dict]], we infer dictionary values types as `List -> Array` and `Dict -> Object`. 
2024/02/05 23:50:21 INFO mlflow.types.utils: MLflow 2.9.0 introduces model signature with new data types for lists and dictionaries. For input such as Dict[str, Union[scalars, List, Dict]], we infer dictionary values types as `List -> Array` and `Dict -> Object`. 


Epoch 1/200

1/1 [==============================] - 10s 10s/step - loss: 0.4094 - val_loss: 0.4034
Epoch 2/200
1/1 [==============================] - 0s 337ms/step - loss: 0.4082 - val_loss: 0.4013
Epoch 3/200
1/1 [==============================] - 0s 358ms/step - loss: 0.4073 - val_loss: 0.3992
Epoch 4/200
1/1 [==============================] - 0s 328ms/step - loss: 0.4064 - val_loss: 0.3971
Epoch 5/200
1/1 [==============================] - 0s 314ms/step - loss: 0.4054 - val_loss: 0.3950
Epoch 6/200
1/1 [==============================] - 0s 325ms/step - loss: 0.4046 - val_loss: 0.3931
Epoch 7/200
1/1 [==============================] - 0s 343ms/step - loss: 0.4038 - val_loss: 0.3911
Epoch 8/200
1/1 [==============================] - 0s 384ms/step - loss: 0.4030 - val_loss: 0.3893
Epoch 9/200
1/1 [==============================] - 0s 371ms/step - loss: 0.4024 - val_loss: 0.3875
Epoch 10/200
1/1 [==============================] - 0s 316ms/step - loss: 0.4017 - val_loss: 0.3858
Epoch 11/

INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmp3f_us4gk\model\data\model\assets


In [23]:
from time import gmtime, strftime

In [24]:
now = datetime.now()

time = now.strftime("%d-%m-%Y-%H-%M-%S-%f")[:-3] # Remove microseconds
model.save(f'models/LSTM_sales-{time}.pkl')

INFO:tensorflow:Assets written to: models/LSTM_sales-05-02-2024-23-52-38-368.pkl\assets


INFO:tensorflow:Assets written to: models/LSTM_sales-05-02-2024-23-52-38-368.pkl\assets
